In [2]:
from bitcoin_deep_learning.call_api import ApiCall
from bitcoin_deep_learning.model import RnnDlModel, LinearRegressionBaselineModel, DummyModel
from bitcoin_deep_learning.trainer import train, read_result
from bitcoin_deep_learning.cross_val import get_cross_XY
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import random
import joblib
%load_ext autoreload
%autoreload 2

ImportError: cannot import name 'cross_val_metrics' from 'bitcoin_deep_learning.cross_val' (/Users/chbro/code/Albore952/bitcoin_deep_learning/bitcoin_deep_learning/cross_val.py)

In [ ]:
#import X_train to tune the model
#intuition too much regularization

In [ ]:
train_df = ApiCall().read_local(data='train')#.dr op(columns = '[%]_Bitcoin_growth_rate_on_Horizon=7')

In [ ]:
X_train_list, Y_train_list, X_test_list,Y_test_list = get_cross_XY()

In [ ]:
np.array(X_train_list).shape

In [ ]:
model = LinearRegressionBaselineModel()

In [ ]:
train(model, train_df)

In [ ]:
coeff_matrix = pd.DataFrame(zip(train_df.columns,model.model.coef_))
coeff_matrix = coeff_matrix.rename(columns={0: "features", 1: "coeff"})
coeff_matrix_absolute = coeff_matrix.copy()
coeff_matrix_absolute['coeff abs'] = coeff_matrix['coeff'].apply(abs)
coeff_matrix_absolute = coeff_matrix_absolute.drop(columns = 'coeff')
coeff_matrix_absolute.sort_values(by = 'coeff abs', ascending = False).reset_index(drop = True)

In [ ]:
plt.figure(figsize = (22,20))
sns.heatmap(round(train_df.corr(),2), cmap = "coolwarm", annot = True, annot_kws = {"size":12});

In [ ]:
train_df.corr().style.background_gradient(cmap='coolwarm')